## Dimension Reduction

#### *11 November 2019*

#### *DATA 1010*

In [1]:
using Pkg; Pkg.activate(".")

Activating environment at `~/Google Drive/DSI/Masters Program/DATA 1010/DATA1010-2019/class-exercises/11-11/Project.toml`


In [2]:
using Plots, Distributions, Images, Interact

## Problem 1

Think of some reasons it would be nice to be able to take a dataset with lots of features and map it to a dataset with very few features, like 2 or 3, while still preserving much of the structure of the original features.

*Solution*. 

## Problem 2

Come up with a scheme for taking a three-dimensional ellipsoidal point cloud and identifying the axes of the ellipsoid.

In [3]:
X = rand(MvNormal([0.0, 2, 1], [3.0 2 1; 2 2 0.5; 1 0.5 0.6]), 1000)'
plotlyjs()
scatter(X[:, 1], X[:, 2], X[:, 3], legend = false, ms = 1, msw = 0.5)

<!DOCTYPE html>
 
 
 Plots.jl

# Problem 3

Another way to map our training points to a lower-dimensional feature space is to try explicitly to preserve some specific structure of the dataset. For example, we can focus on which points are close to which points by connecting each point to the 3 points closest to it in the training set, and then treating the points as repelling particles and the connections as springs. Visit [Chris Olah's blog](https://colah.github.io/posts/2014-10-Visualizing-MNIST/) to see this approach in action (you can search "Visualizing MNIST as a Graph" to get there quickly). Note that there are both 2D and 3D examples shown there.

Look for a point in that dataset which sits among points of a different class. Mouse over it to see the image corresponding to that point; does it look more similar to the points it's clustered with than some other images of its class?

---

A more popular dimension reduction technique takes the same general approach but is smoother and typically more effective at identifying structure in the dataset. 

## Problem 4 

Let $P_{i,j}(\sigma) = \frac{\operatorname{e}^{-|\mathbf{x}_i -
      \mathbf{x}_j|^2/(2\sigma^2)}}{\sum_{k\neq j} \operatorname{e}^{-|\mathbf{x}_k
      - \mathbf{x}_j|^2/(2\sigma^2)}}$, if $i$ and $j$ are row indices of the feature matrix $X$. Plot the perplexity $2^{-\sum_{i : i \neq j} P_{i,j}(\sigma) \log_2 P_{i,j}(\sigma)}$ as a function of $\sigma$ for the tiny dataset below. 

In [4]:
x = [[0,0],[0,1],[1,1],[4,0]]
f(x,y,σ) = exp(-norm(x-y)^2/(2σ^2))
P(x,i,j,σ) = f(x[i],x[j],σ) / sum(f(x[k],x[j],σ) for k in 1:length(x) if k ≠ j)
[P(x,2,1,σ) for σ in [0.25, 1, 2, 100]]

4-element Array{Float64,1}:
 0.9996646498695336
 0.6222451093055732
 0.4911948727783495
 0.3334222145344327

---

## Problem 5 (Eigenfaces)

First, let us load the dataset. It consists of $165$ images of faces ($231 \times 195$ pixels) obtained from $15$ individuals. There are $11$ image per subject, consisting of different lighting conditions (illuminated from center, left or right), facial expressions (normal, sad, happy, sleepy, surprised or winking) and with or without glasses.

In [5]:
images = [float.(load("yale_faces/" * file)) for file in readdir("yale_faces") if endswith(file,".pgm")];

function imagedisplay(img)
    newimg = img / maximum(abs.(img))
    # if half or more of the pixel values are negative, flip the sign
    if mean(newimg .> 0) < 1/2
        sgn = -1
    else
        sgn = 1
    end
    Gray.(sgn .* newimg)
end

imagedisplay (generic function with 1 method)

In [6]:
@manipulate for k in 1:165
    imagedisplay(images[k])
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["k"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 165,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Int64} with 2 listeners. Value:
83, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/Users/sswatson/.julia/packages/Knockout/1sDlc/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/sswatson/.julia/packages/Knockout/1sDlc/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/sswatson/.julia/packages/InteractBase/9mFwe/src/../assets/all.js"), Asset("css", nothing, "/Users/sswatson/.julia/packages/InteractBase/9mFwe/src/../assets/style.css"), Asset("css", nothing, "/Users/sswatson/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x0000000119b12650, Task (runnable) @0x0000000119b12650), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1\",\"2\",\"3\",\"4\",\"5\",\"6\",\"7\",\"8\",\"9\",\"10\",\"11\",\"12\",\"13\",\"14\",\"15\",\"16\",\"17\",\"18\",\"19\",\"20\",\"21\",\"22\",\"23\",\"24\",\"25\",\"26\",\"27\",\"28\",\"29\",\"30\",\"31\",\"32\",\"33\",\"34\",\"3

(a) Show the mean of all 165 images.

(b) Use Interact to look at the first k principal components.

(c) Make a scatter plot of the original data points projected onto the span of the first two principal components. 

In [7]:
V[:,1:2]'

2×45045 Adjoint{Float64,Array{Float64,2}}:
 -1.21842e-5  1.1776e-5    0.000180459  …   0.00639419   0.00541057
  0.00062372  0.000736748  0.00151792      -0.003972    -0.00410024

In [8]:
#V[:,1:2] * (V[:,1:2]' * bigimagemat[1,:])
#V[:,1:2] * (V[:,1:2]' * bigimagemat[1,:])